In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy import stats
import os

# Shutil is a recursive tool to delete file paths
# Note: Only needed if you plan to overwrite existing filepaths
import shutil

data1 = pd.read_excel('Batch_Div27_Class8_2_Results_metrics_BEV.xlsx',index_col=None)
# data2 = pd.read_excel('Div27_Class6_1_Results_metrics_BEV.xlsx',index_col=None)
# data = pd.concat([data1, data2])
# data.info()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3876 entries, 0 to 1139
Columns: 104 entries, RunId-none to Characteristic Acceleration-mph/s
dtypes: float64(77), int64(21), object(6)
memory usage: 3.1+ MB


In [3]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2736 entries, 0 to 2735
Columns: 103 entries, RunId-none to Accessory work-kJ
dtypes: float64(75), int64(22), object(6)
memory usage: 2.2+ MB


In [4]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Columns: 104 entries, RunId-none to Accessory work-kJ
dtypes: float64(77), int64(21), object(6)
memory usage: 926.4+ KB


In [11]:
data1.head()

,RunId-none,data_set-none,Vehicle Model-none,Cycle Name-none,Cycle Source-none,Fuel Type-none,Vehicle Static Mass-lbm,Cargo Mass-lbm,Transmission-none,Axle ratio-none,...,Total No. of shifts on uphill per 100 miles-1/100mile,Percent time when wheel brakes are ON-%,Time spent in motoring-s,Distance traveled in motoring-mile,Time spent in motoring w/o wheel brakes-s,Distance traveled in motoring w/o wheel brakes-mile,Time spent in motoring w/o engine brakes-s,Distance traveled in motoring w/o engine brakes-mile,Fuel energy consumed-kJ,Accessory work-kJ
0,2281,2021_0803_0539_21_626-24,class8_bev_2021_low_sleeper_longhaul_FU19,Fusion_HERE_Route_From_Mat_Dedi_1666_div27_1_2...,Dedi_1666_div27_1_2_Raw,Electricity,35048.016720,0.0,AMT,26,...,8.449812,24.049361,0,0,0,0,0,0,0,62817.30
1,2282,2021_0803_0539_32_841-35,class8_bev_2021_low_sleeper_longhaul_FU19,Fusion_HERE_Route_From_Mat_Dedi_1666_div27_1_2...,Dedi_1666_div27_1_2_Raw,Electricity,35048.016720,0.0,AMT,26,...,8.449812,24.049361,0,0,0,0,0,0,0,62817.30
2,2283,2021_0803_0539_28_478-36,class8_bev_2021_low_sleeper_longhaul_FU19,Fusion_HERE_Route_From_Mat_Dedi_1666_div27_1_2...,Dedi_1666_div27_1_2_Raw,Electricity,35048.016720,0.0,AMT,26,...,8.449812,24.049361,0,0,0,0,0,0,0,62817.30
3,2284,2021_0803_0539_30_627-75,class8_bev_2021_low_sleeper_longhaul_FU19,Fusion_HERE_Route_From_Mat_Dedi_1666_div27_1_2...,Dedi_1666_div27_1_2_Raw,Electricity,48947.953227,0.0,AMT,26,...,9.294736,25.240839,0,0,0,0,0,0,0,62768.92
4,2285,2021_0803_0539_33_445-27,class8_bev_2021_low_sleeper_longhaul_FU19,Fusion_HERE_Route_From_Mat_Dedi_1666_div27_1_2...,Dedi_1666_div27_1_2_Raw,Electricity,48947.953227,0.0,AMT,26,...,9.294736,25.240839,0,0,0,0,0,0,0,62768.92


In [37]:
data_learn0 = data1[['Vehicle Model-none',\
            'Battery Energy Consumption per Mile-kWh/mi',\
            'Vehicle Static Mass-lbm',\
            'Cargo Mass-lbm']]
train_data = data_learn0[['Cargo Mass-lbm']]*0.453592
train_data.columns = ['CargoMass_kg']
train_data['Energy_kwh_km'] = data_learn0[['Battery Energy Consumption per Mile-kWh/mi']]/1.60934
# data_learn = pd.get_dummies(data_learn0,drop_first=True)
# data_learn

In [42]:
train0 = pd.DataFrame(data_learn.iloc[:,1:2])*0.453592
# train0.columns = ['Vehiecle Mass_kg','CargoMass_kg']
train0.columns = ['Mass_kg']
train0['Energy_kwh_km'] = data_learn.iloc[:,0]/1.60934
type(data_learn.iloc[:,0])

pandas.core.series.Series

In [ ]:
#LR model
import statsmodels.formula.api as smf
import statsmodels.api as sm
# fit the data to the model
# X_train1 = sm.add_constant(X_train)
# LR = sm.OLS(y_train, X_train1).fit()

LR = smf.ols(formula='Energy_kwh_km ~ Mass_kg ',data=train0).fit()
print(LR.summary())

In [ ]:
Task: 
    Create a lookup table for the linear cost constants (a, b), 
    and save it as a csv of pickle file
input:
    - bool freight = Are we using 
    - double m is the freight mass
    - int i is the node that the vehiecle is departing from
    - int j is the node that the vehiecle's destination is 
    - int k is the identification number of the vehiecle
returns:
    - (a, b) as the results of linear regression from the data
    - 1. am+b, m 是 freight mass, a_ijk, b_ijk
    - 2. am+b, m 是 vehicle mass, a_ijk, b_ij

def getCostConstants(freight = True, m, i, j, k):
    if freight:
        # case where m = freight mass
        
        
    else:
        # case where m = vehicle mass
                

In [55]:
ts = pd.Series(data1['Vehicle Model-none'].values).unique()

# helper function to find the numbers in a string
df = data1.loc[:, ['Vehicle Model-none',\
        'Battery Energy Consumption per Mile-kWh/mi',\
        'Vehicle Static Mass-lbm',\
        'Cargo Mass-lbm']]

def findInt(sentence_arr):
    running_max = 1
    length = len(sentence_arr)
    i_val = np.full(length, 0)
    j_val = np.full(length, 0)
    for index, sentence in enumerate(sentence_arr):
        digits = [int(i) for i in sentence if i.isdigit()] 
        running_max = max(digits[-1], digits[-2], running_max)
        i_val[index], j_val[index] = digits[-2], digits[-1]
    return running_max, i_val, j_val
numNodes, i_val, j_val = findInt(data1["Cycle Source-none"].values)
i_val, j_val = pd.Series(i_val), pd.Series(j_val)
df['i'], df['j'] = i_val, j_val
df

,Vehicle Model-none,Battery Energy Consumption per Mile-kWh/mi,Vehicle Static Mass-lbm,Cargo Mass-lbm,i,j
0,class8_bev_2021_low_sleeper_longhaul_FU19,1.668647,35048.016720,0.000000,1,2
1,class8_bev_2021_low_sleeper_longhaul_FU19,1.668350,35048.016720,0.000000,1,2
2,class8_bev_2021_low_sleeper_longhaul_FU19,1.670148,35048.016720,0.000000,1,2
3,class8_bev_2021_low_sleeper_longhaul_FU19,1.865797,48947.953227,0.000000,1,2
4,class8_bev_2021_low_sleeper_longhaul_FU19,1.865496,48947.953227,0.000000,1,2
...,...,...,...,...,...,...
2731,class8_bev_2021_daycab_regionalhaul_FU19,2.186213,67711.070742,36867.052329,4,3
2732,class8_bev_2021_daycab_regionalhaul_FU19,2.429584,67711.070742,36867.052329,4,3
2733,class8_bev_2021_daycab_regionalhaul_FU19,2.351021,80000.088185,49156.951622,4,3
2734,class8_bev_2021_daycab_regionalhaul_FU19,2.406699,80000.088185,49156.951622,4,3


# Function: generateData
### Creates the lognormal Sampled data and stores it in csv files. 

> (str) dataSourcePath : csv data file path  
(str) saveFolderName : Name of the folder to save the data (non-existing)
(int) days : # of days
(float) mean multiplier for the lognormal standard deviation  
(int) randomSeed : andom seed for the lognormal Sampling process


#### Note:  -1 is a placeholder for N/A values (self loops) and -2 is used as padding (for lookup convenience) 


In [109]:
# This function will store the sampled data into multiple csv files.
def generateData (freight = True, dataSourcePath = 'Batch_Div27_Class8_2_Results_metrics_BEV.xlsx', saveFolderName = "ABData"):
    parent_dir = os.getcwd()
    savePath = os.path.join(parent_dir, saveFolderName)
    
    # If the folder already exists, remove the folder
    if os.path.exists(savePath):
        shutil.rmtree(savePath)
    os.makedirs(savePath)
    
    # Load and trim Data
    df = pd.read_excel(dataSourcePath, index_col=None)
    df = df.loc[:, ['Vehicle Model-none',\
            'Battery Energy Consumption per Mile-kWh/mi',\
            'Vehicle Static Mass-lbm',\
            'Cargo Mass-lbm']]
    
    # Getting information from the table
    vehicleTypes = pd.Series(data1['Vehicle Model-none'].values).unique()
    # Function to look for the number of nodes in a list of strings
    # Should also add columns to re-label the i and j-s into the main dataframe for lookup convenience later in this code
    def findInt(sentence_arr):
        running_max = 1
        length = len(sentence_arr)
        i_val = np.full(length, 0)
        j_val = np.full(length, 0)
        for index, sentence in enumerate(sentence_arr):
            digits = [int(i) for i in sentence if i.isdigit()] 
            running_max = max(digits[-1], digits[-2], running_max)
            i_val[index], j_val[index] = digits[-2], digits[-1]
        return running_max, i_val, j_val
    numNodes, i_val, j_val = findInt(data1["Cycle Source-none"].values)
    i_val, j_val = pd.Series(i_val), pd.Series(j_val)
    df['i'], df['j'] = i_val, j_val
    
    # Setup for labels
    cols = ["i_" + str(i) for i in range(1, numNodes+1)]
    rows = ["j_" + str(i) for i in range(1, numNodes+1)]
                   
    # Creating Vehicle File paths
    for v in vehicleTypes:
        workFrame = df[df["Vehicle Model-none"] == v]
        newFrame = pd.DataFrame(columns = cols, index = rows)

        # Iterate over node values.
        for i in range(1, numNodes + 1):
            for j in range(1, numNodes + 1):
                # This line below gives a frame of the cost of an edge
                data = workFrame.loc[((workFrame['i'] == i) & (workFrame['j'] == j))]

                # edge case if i = j or data Doesn't exist
                if i == j :
                    newFrame.iat[i-1, j-1] = -1
                    continue
                if data.size == 0:
                    newFrame.iat[i-1, j-1] = 0
                    continue
                    
                # Doing linear regression here
                if freight:
                    # Set training data to be Cargo data
                    train_data = data.loc[:, ['Cargo Mass-lbm']]*0.453592
                    train_data.columns = ['CargoMass_kg']
                    train_data['Energy_kwh_km'] = data.loc[:, ['Battery Energy Consumption per Mile-kWh/mi']]/1.60934
                    LR = smf.ols(formula='Energy_kwh_km ~ CargoMass_kg ',data=train_data).fit()
                else:
                    train_data = data.loc[:, ['Vehicle Static Mass-lbm']]*0.453592
                    train_data.columns = ['VehicleMass_kg']
                    train_data['Energy_kwh_km'] = data.loc[:, ['Battery Energy Consumption per Mile-kWh/mi']]/1.60934
                    LR = smf.ols(formula='Energy_kwh_km ~ VehicleMass_kg ',data=train_data).fit()
                
                # Assigning the calculated a, b params
                newFrame.iat[i - 1, j - 1] = (LR.params[0], LR.params[1])

        # Create the filePath
        tempPath2 = os.path.join(savePath, v + ".csv")
        tempPath3 = os.path.join(savePath, v + ".json")
        newFrame.to_csv(tempPath2)
        newFrame.to_json(tempPath3)

In [110]:
generateData(freight = True, dataSourcePath = 'Batch_Div27_Class8_2_Results_metrics_BEV.xlsx', saveFolderName = "ABData")

# Function: readData
 ### Reads the source folder generated by the  "generateData" function and stores it into a 5-dimensioned dictionary.
> Input:  
(str) folderName: Folder Name of the source folder
  
Output: 
> arr [ v ][ d ][ k ][ i ][ j ]

>v : Vehiecle Type (string)  
d : Day   (int), index starts from 1  
k : State (int), index starts from 1  
i : Node  (int), index starts from 1  
j : Node  (int), index starts from 1  

#### Note:  -1 is a placeholder for N/A values (self loops) and -2 is used as padding (for lookup convenience) 

In [149]:
# Reads out the data and collects them into a dictionary.
def readDataCSV (folderName = "ABData"):
    # Initialize some information
    path = os.path.join(os.getcwd(), folderName)
    # If the folder doesn't exist, print error and return
    if not os.path.exists(path):
        print("Error, Source Folder Doesn't exist")
        return 
    VehiecleTypesPath = [ fi for fi in os.listdir(path) if fi.endswith(".json") ]
    VehiecleTypes = [i[:len(i) - 5] for i in VehiecleTypesPath]
    
    # Get the number of nodes brute forcedly
    filePath = os.path.join(path, VehiecleTypesPath[0])
    df = pd.read_json(filePath).iloc[: , 1:]
    numNodes = np.shape(df.iloc[:, 0].values)[0]
    
    # Return arr
    a = {}
    b = {}

    for ind, v in enumerate(VehiecleTypes):
        # initialize an empty array to store the days
        a[v] = np.full((numNodes + 1, numNodes + 1), -1, dtype = np.float64)
        b[v] = np.full((numNodes + 1, numNodes + 1), -1, dtype = np.float64)
        
        results = df.to_numpy().transpose()
        for i, temp in enumerate(results, 1):
            for j, ab in enumerate(temp, 1):
                if type(ab) == list:
                    a[v][i][j], b[v][i][j] = ab[0], ab[1] 
                else:
                    a[v][i][j], b[v][i][j] = ab, ab
                    
    return a, b

In [150]:
a, b = readDataCSV()

In [152]:
b

{'class8_bev_2021_daycab_regionalhaul_FU19': array([[-1.00000e+00, -1.00000e+00, -1.00000e+00, -1.00000e+00,
         -1.00000e+00, -1.00000e+00, -1.00000e+00],
        [-1.00000e+00,  2.74195e-05, -1.00000e+00,  2.59624e-05,
          0.00000e+00,  0.00000e+00,  0.00000e+00],
        [-1.00000e+00,  0.00000e+00,  2.49329e-05, -1.00000e+00,
          2.65268e-05,  0.00000e+00,  0.00000e+00],
        [-1.00000e+00,  0.00000e+00,  0.00000e+00,  2.49375e-05,
         -1.00000e+00,  0.00000e+00,  0.00000e+00],
        [-1.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
          0.00000e+00, -1.00000e+00,  2.63563e-05],
        [-1.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
          0.00000e+00,  2.51175e-05, -1.00000e+00],
        [-1.00000e+00, -1.00000e+00, -1.00000e+00, -1.00000e+00,
         -1.00000e+00, -1.00000e+00, -1.00000e+00]]),
 'class8_bev_2021_low_sleeper_longhaul_FU19': array([[-1.00000e+00, -1.00000e+00, -1.00000e+00, -1.00000e+00,
         -1.00000e+00, -1

In [12]:
# Example lookup
testarr['daycab_regionalhaul'][4][5][2][1]

2.04703264333656